In [22]:
import pickle
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F

In [14]:
params = pd.read_pickle("./train_model_5a04d_00095_95_batch_size=512,hidden_size=64,learning_rate=0.0003,max_epochs=1690,n_steps_in=6,n_steps_out=10,num_laye_2023-05-04_18-44-24\params.pkl")
params

{'n_steps_in': 6,
 'n_steps_out': 10,
 'hidden_size': 64,
 'learning_rate': 0.00026714846723588354,
 'weight_decay': 1.2462812380347419e-06,
 'batch_size': 512,
 'max_epochs': 1690,
 'num_layers': 1,
 'input_size': 2,
 'checkpoint_freq': 10,
 'max_rows': None,
 'fast_dev_run': False,
 'gpu': 1,
 'include_indicator': True,
 'delay_in_the_indicator': 0,
 'gap': 1,
 'include_target': True,
 'num_samples': 100,
 'skiprows': 120000}

In [15]:
with open('pickle.dat', 'rb') as file:
    # Load the object from the file
    data = pickle.load(file)

In [17]:
len(data)

2

In [18]:
data[0].shape, data[1].shape

(torch.Size([1617466, 6, 2]), torch.Size([1617466, 10]))

In [19]:
data[1][:5, :]

tensor([[0.0902, 0.0902, 0.0902, 0.0902, 0.0902, 0.0902, 0.0902, 0.0910, 0.0910,
         0.0910],
        [0.0902, 0.0902, 0.0902, 0.0902, 0.0902, 0.0902, 0.0910, 0.0910, 0.0910,
         0.0910],
        [0.0902, 0.0902, 0.0902, 0.0902, 0.0902, 0.0910, 0.0910, 0.0910, 0.0910,
         0.0910],
        [0.0902, 0.0902, 0.0902, 0.0902, 0.0910, 0.0910, 0.0910, 0.0910, 0.0910,
         0.0910],
        [0.0902, 0.0902, 0.0902, 0.0910, 0.0910, 0.0910, 0.0910, 0.0910, 0.0910,
         0.0910]])

In [20]:
data[0][0]

tensor([[0.0019, 0.0894],
        [0.0019, 0.0894],
        [0.0019, 0.0894],
        [0.0019, 0.0894],
        [0.0019, 0.0894],
        [0.0019, 0.0902]])

In [53]:
# Ref: https://www.kaggle.com/code/sagarikajadon/gb-vpp-pytorch-lstm-baseline
class BiLSTM(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        hidden_dim= [400, 300, 200, 100]
        self.bilstm1= nn.LSTM(input_dim, hidden_dim[0], batch_first= True, bidirectional= True)
        self.norm1= nn.LayerNorm(hidden_dim[0]*2)
        
        self.bilstm2= nn.LSTM(hidden_dim[0]*2, hidden_dim[1], batch_first= True, bidirectional= True)
        self.norm2= nn.LayerNorm(hidden_dim[1]*2)
        
        self.bilstm3= nn.LSTM(hidden_dim[1]*2, hidden_dim[2], batch_first= True, bidirectional= True)
        self.norm3= nn.LayerNorm(hidden_dim[2]*2)
        
        self.bilstm4= nn.LSTM(hidden_dim[2]*2, hidden_dim[3], batch_first= True, bidirectional= True)
        self.norm4= nn.LayerNorm(hidden_dim[3]*2)
        
        self.fc1= nn.Linear(hidden_dim[3]*2, 100)
        self.fc2= nn.Linear(100, output_dim)

        
    def forward(self, X):
        pred, _= self.bilstm1(X)
        pred= self.norm1(pred)
        
        pred, _= self.bilstm2(pred)
        pred= self.norm2(pred)
        
        pred, _= self.bilstm3(pred)
        pred= self.norm3(pred)
        
        pred, _= self.bilstm4(pred)
        pred= self.norm4(pred)
        
        pred= self.fc1(pred[:, -1, :])
        pred= F.selu(pred)
        
        pred= self.fc2(pred)
        return pred

In [58]:
model = BiLSTM(2, 10)

In [59]:
x = torch.rand(32, 6, 2)

model(x).shape

torch.Size([32, 6, 200])


torch.Size([32, 10])